In [1]:
from requests import get, post
from web3 import Web3, HTTPProvider
from icecream import ic
import numpy as np
import pandas as pd
import sys
from dotenv import load_dotenv
import os
import math
import json
import matplotlib.pyplot as plt


In [2]:
# Add the directory containing the module to the Python path
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

# Import functions from the module
from uniV3Pricing import get_account_data, get_uniswap_slot0, get_nft_positions_details, get_amounts_from_ticks

In [3]:
# Explicitly specify the path to .env file using forward slashes
load_dotenv(dotenv_path='.env')

# Verify that the environment variable is loaded correctly
alchemy_key = os.getenv('ALCHEMY_KEY')

# Create the Web3 provider using the Alchemy URL
w3 = Web3(HTTPProvider(f"https://base-mainnet.g.alchemy.com/v2/{alchemy_key}")) # BASE
# w3 = Web3(HTTPProvider(f"https://eth-mainnet.g.alchemy.com/v2/{alchemy_key}")) # ETHEREUM

# Check connection
print(f"Web3 is connected: {w3.is_connected()}")


Web3 is connected: True


### GraphQL Queries

In [4]:
# Define GraphQL queries
query_accounts = """
query MyQuery {
  allAccountAssets {
    account
    numeraire
    debtUsd
    liquidationValue
    healthy
    assetDetails
    assetDetailsUsd
    collateralValue
    collateralValueUsd
    positionDistribution
    positionDistributionUsd
    usdcValue
    usedMargin
    wethValue
  }
}
"""

### Define helper functions

In [5]:
def fetch_data(url, query):
    response = post(url, json={'query': query})
    if response.status_code == 200:
        return response.json()['data']
    else:
        raise Exception(f"Query failed to run by returning code of {response.status_code}. {query}")

def safe_json_loads(val):
    if val is None:
        return {}
    try:
        return json.loads(val.replace("'", "\""))
    except json.JSONDecodeError:
        return {}


### Fetch Data

In [6]:
url = "https://data-core.up.railway.app/protocols/arcadia/graphql/"

# Fetch account data
account_data = fetch_data(url, query_accounts)['allAccountAssets']

### Process Account Data

In [ ]:
# Convert JSON data to a DataFrame
accounts_df = pd.DataFrame(account_data)

# Parse nested JSON fields
accounts_df['assetDetails'] = accounts_df['assetDetails'].apply(safe_json_loads)
accounts_df['assetDetailsUsd'] = accounts_df['assetDetailsUsd'].apply(safe_json_loads)
accounts_df['positionDistribution'] = accounts_df['positionDistribution'].apply(safe_json_loads)
accounts_df['positionDistributionUsd'] = accounts_df['positionDistributionUsd'].apply(safe_json_loads)

# Ensure 'debtUsd' is treated as a float and filter for non-zero debtUsd
accounts_df['debtUsd'] = accounts_df['debtUsd'].astype(float)
debtUsd_df = accounts_df[accounts_df['debtUsd'] > 0]

# Display DataFrame
print("Accounts Data:")
print(debtUsd_df.head())


In [8]:
# Extract keys from positionDistribution where the value is 0
keys_with_zero_value = []

for distribution in debtUsd_df['positionDistribution']:
    keys_with_zero_value.extend([key for key, value in distribution.items() if value == 0])

# Get a list of unique keys
unique_keys_with_zero_value = list(set(keys_with_zero_value))

# Display the unique keys
print("Unique keys with zero value in positionDistribution:")
print(unique_keys_with_zero_value)


Unique keys with zero value in positionDistribution:
['0x17b5826382e3a5257b829cf0546a08bd77409270', '0x9f42361b7602df1a8ae28bf63e6cb1883cd44c27', '0x1dc7a0f5336f52724b650e39174cfcbbedd67bf1']


In [9]:
from collections import Counter

# Extract keys from positionDistribution where the value is 0
keys_with_zero_value = []

for distribution in debtUsd_df['positionDistribution']:
    keys_with_zero_value.extend([key for key, value in distribution.items() if value == 0])

# Get the count of occurrences of each unique key
key_counts = Counter(keys_with_zero_value)

# Display the unique keys and their counts
print("Unique keys with zero value and their occurrences in positionDistribution:")
for key, count in key_counts.items():
    print(f"{key}: {count}")


Unique keys with zero value and their occurrences in positionDistribution:
0x1dc7a0f5336f52724b650e39174cfcbbedd67bf1: 70
0x17b5826382e3a5257b829cf0546a08bd77409270: 1
0x9f42361b7602df1a8ae28bf63e6cb1883cd44c27: 2


In [12]:
# Define the ID to search for
search_id = "0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1"

# Initialize an empty list to store the results
results = []

# Iterate through the assetDetails column
for asset_detail in debtUsd_df['assetDetails']:
    for sublist in asset_detail:
        if search_id in sublist:
            index = sublist.index(search_id)
            results.append([search_id, asset_detail[1][index]])

# Convert the results to a DataFrame
results_df = pd.DataFrame(results, columns=['id', 'value'])

# Display the resulting DataFrame
print(results_df)

# Create a list of elements in the 'value' column
value_list = results_df['value'].tolist()
print(value_list)

                                            id   value
0   0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1  208636
1   0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1  210085
2   0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1  161277
3   0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1  203125
4   0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1  232258
..                                         ...     ...
65  0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1  234344
66  0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1  234435
67  0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1  236198
68  0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1  237190
69  0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1  241770

[70 rows x 2 columns]
[208636, 210085, 161277, 203125, 232258, 210105, 206932, 241524, 214760, 242900, 201416, 202677, 229419, 243115, 220056, 221905, 228199, 207014, 227759, 216102, 216130, 207895, 207899, 221929, 218757, 217420, 204057, 206498, 233148, 235799, 195163, 195094, 195546, 238969, 195701, 195889, 236958, 198061, 198302, 2000